### Codebook Processing

#### Pull the codebook text file into a Pandas dataframe:  
    cbk = pd.read_table(path+filename, sep='\\s+', lineterminator = '\n', header=None)  
    cbk.columns = ['variable','basis','range','length']  
      
#### Process the range field into start/end indexes and slice objects  
    a = cbk['range'].str.replace('-',',').str.split(',').tolist()  
    b = [[int(i) for i in j] for j in a]  
    c = []  
    d = []  
    sl = []  
    for i in b:  
        i[0] = i[0]-1  
        c.append(i)  
    print(c)  
    for i in c:  
        if len(i) == 1: i.append(i[0]+1)  
        s = slice(i[0],i[1])    
        sl.append(s)  
    print(sl)  
    print(len(cbk), len(c),len(d), len(sl))   
  
#### Iterate through the list of slice objects and apply to fixed-width text (resulting in a list of text pieces which are data items))
    tx = 'ABCDEFGHIJKLMNOPQRSTUVWXYZABCDEFGHIJKLMNOPQRSTUVWXYZABCDEFGHIJKLMNOPQRSTUVWXYZABCDEFGHIJKLMNOPQRSTUVWXYZABCDEFGHIJKLMNOPQRSTUVWXYZ'  
    limit = len(tx)  
    fields = []  
    for i in sl:  
        len_slice = len(tx[i])  
        begin = limit  
        limit = limit - len_slice  
        if limit <= 0: break  
        print(begin, len_slice, limit, tx[i])  
        fields.append(tx[i])  
    print(fields)  
  
#### Compile and save Pandas dataframe
#### Pickle list of slice objects for use in datafile reading

Initialize Libraries and Global Variables

In [2]:
import re
import pandas as pd
import pickle

path = r'D:\\Projects\\ipums\\garrard_test\\'
filename = r'ky_garrard_1870_field_lengths.txt'
cbkname = r'ky_1870_cbk.csv'
cbkpickle = r'ky_1870_cbk.pickle'
sliceobjs = r'ky_1870_slice_objects.pickle'

csvfile = path + cbkname
cbkpickle = path + cbkpickle
slicepickle = path + sliceobjs

Read the codebook text file

In [3]:
cbk = pd.read_table(path+filename, sep='\\s+', lineterminator = '\n', header=None)
cbk.columns = ['variable','basis','range','length']
cbk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   variable  88 non-null     object
 1   basis     88 non-null     object
 2   range     88 non-null     object
 3   length    88 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 2.9+ KB


Process Codebook Range column into Slice Objects

In [4]:
#Range processing (to list of slices)
a = cbk['range'].str.replace('-',',').str.split(',').tolist()
b = [[int(i) for i in j] for j in a]
c = []
d = []
sl = []
for i in b:
    i[0] = i[0]-1
    c.append(i)
print(c)
for i in c:
    if len(i) == 1: i.append(i[0]+1)
    s = slice(i[0],i[1])  
    sl.append(s)
print(sl)
print(len(cbk), len(c),len(sl))

[[0, 4], [4, 12], [12, 14], [14, 16], [16, 20], [20], [21, 23], [23, 25], [25, 27], [27, 31], [31, 38], [38], [39, 42], [42, 46], [46, 50], [50, 57], [57, 59], [59], [60, 67], [67, 74], [74, 77], [77], [78], [79, 82], [82, 84], [84], [85, 87], [87], [88], [89], [90], [91, 99], [99, 103], [103, 139], [139, 143], [143, 145], [145, 147], [147, 149], [149], [150], [151, 153], [153], [154], [155, 157], [157], [158], [159], [160], [161, 163], [163, 165], [165, 167], [167, 171], [171, 173], [173], [174, 177], [177, 179], [179, 181], [181, 185], [185], [186], [187, 190], [190], [191, 194], [194, 197], [197, 202], [202], [203], [204], [205], [206], [207], [208, 212], [212, 215], [215, 218], [218, 226], [226, 234], [234, 236], [236, 272], [272, 274], [274, 295], [295], [296], [297], [298], [299], [300], [301], [302, 309]]
[slice(0, 4, None), slice(4, 12, None), slice(12, 14, None), slice(14, 16, None), slice(16, 20, None), slice(20, 21, None), slice(21, 23, None), slice(23, 25, None), slice(25, 

In [5]:
tx = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
limit = len(tx)
fields = []
for i in sl:
    len_slice = len(tx[i])
    begin = limit
    limit = limit - len_slice
    if limit <= 0: break
    print(begin, len_slice, limit, tx[i])
    fields.append(tx[i])
print(fields)

26 4 22 ABCD
22 8 14 EFGHIJKL
14 2 12 MN
12 2 10 OP
10 4 6 QRST
6 1 5 U
5 2 3 VW
3 2 1 XY
['ABCD', 'EFGHIJKL', 'MN', 'OP', 'QRST', 'U', 'VW', 'XY']


Finalize and save the codebook (dataframe)

In [10]:
dfx = pd.DataFrame(c)
dfx.columns = ['start_idx','end_idx']
dfz = pd.DataFrame(sl)
dfz.columns = ['slice_obj']
dfy = pd.concat([cbk,dfx[['start_idx','end_idx']],dfz[['slice_obj']]], axis=1)
print(dfy[20:24])
dfy.to_pickle(cbkpickle)

   variable basis  range  length  start_idx  end_idx            slice_obj
20    CNTRY     H  75-77       3         74       77  slice(74, 77, None)
21       GQ     H     78       1         77       78  slice(77, 78, None)
22   GQTYPE     H     79       1         78       79  slice(78, 79, None)
23  GQTYPED     H  80-82       3         79       82  slice(79, 82, None)


In [11]:
cbk2 = pd.read_pickle(cbkpickle)
a = cbk2['slice_obj'].tolist()
print('column slice_obj is ',type(a[0]),'\n')
print(cbk2[20:24])

column slice_obj is  <class 'slice'> 

   variable basis  range  length  start_idx  end_idx            slice_obj
20    CNTRY     H  75-77       3         74       77  slice(74, 77, None)
21       GQ     H     78       1         77       78  slice(77, 78, None)
22   GQTYPE     H     79       1         78       79  slice(78, 79, None)
23  GQTYPED     H  80-82       3         79       82  slice(79, 82, None)
